In [1]:
import folium
import webbrowser
from folium.plugins import HeatMap

In [19]:
import h3
import pandas as pd

In [ ]:
df311[hex_col] = df311.apply(lambda x: h3.geo_to_h3(x.lat,x.lng,APERTURE_SIZE),1)

In [4]:
import os
cwd = os.getcwd()

In [5]:
os.chdir(os.path.join(cwd,'../..'))
import ExMAS.main
import ExMAS.utils

In [6]:
from ExMAS.utils import inData as inData

In [7]:
params = ExMAS.utils.get_config('ExMAS/data/configs/default.json') # load the default 

In [8]:
inData = ExMAS.utils.load_G(inData, params, stats=True)  # download the graph

In [354]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('/Users/rkucharski/Documents/GitHub/Shareability')
from reliability import *
from corona import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import pandas as pd
pd.options.mode.chained_assignment = None
pd.options.display.float_format = '{:.3f}'.format
import numpy as np
from math import *
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import json
from scipy.stats import *

import pprint

In [4]:
params = ExMAS.utils.get_config('ExMAS/data/configs/default.json') # load the default 

In [10]:
inData = ExMAS.utils.load_G(inData, params, stats=True)

In [14]:
inData.requests = pd.read_csv('ExMAS/data/corona/results/requests.csv')
#inData.sblts.requests = pd.read_csv('../data/corona/sblt_requests.csv')
inData.sblts.rides = pd.read_csv('ExMAS/data/corona/results/rides.csv')
inData.passengers = pd.read_csv('ExMAS/data/corona/results/passengers.csv')
for col in ['times', 'indexes', 'u_paxes', 'indexes_orig', 'indexes_dest']:
    inData.sblts.rides[col] = inData.sblts.rides[col].apply(lambda x: json.loads(x))
inData.passengers.platforms = inData.passengers.platforms.apply(lambda x: json.loads(x))

In [15]:
# inData.sblts.rides = rides
# inData.requests = requests
# inData.sblts.requests = requests
# inData.sblts.schedule = schedule
CENTER = [52.3703923, 4.877417]

In [16]:
def res_reader(path = '../data/corona/results'):
    evolutions = list()
    last_days = list()
    populations = list()
    for filename in os.listdir(path):
        if filename.startswith('corona') or filename.startswith('population'):
            df = pd.read_csv(os.path.join(path,filename), index_col = 0)
            header = dict()
            for h in filename[:-4].split("_")[1:]:
                df[h.split("-")[0]]=float(h.split("-")[1])
            if filename.startswith('corona'):                
                df.index.name = 'day'
                last_days.append(pd.DataFrame(df.iloc[-1]).T)
                df = df.reindex(range(900)).fillna(method = 'ffill') 
                df['day'] = df.index.astype(int)              
                evolutions.append(df)
            elif filename.startswith('population'):
                populations.append(df)
    evolutions = pd.concat(evolutions)
    for col in ['I','Q','R','S','day', 'nP' , 'repl']:
        evolutions[col] = evolutions[col].fillna(0)
        evolutions[col] = evolutions[col].astype(int)
    populations = pd.concat(populations)
    last_days = pd.concat(last_days)
    last_days['day'] = last_days.index.astype(int)   
    for col in ['I','Q','R','S','day', 'nP' , 'repl']:
        last_days[col] = last_days[col].fillna(0)
        last_days[col] = last_days[col].astype(int)
    
    return evolutions, populations, last_days

evolutions, populations, last_days = res_reader()

FileNotFoundError: [Errno 2] No such file or directory: '../data/corona/results'

In [197]:
def get_infection_map(path = "res/population_3000_init_0.025_repl_1584525041.2557492.csv"):
    df = pd.read_csv(path)
    del df['Unnamed: 0']
    inData.population = df
    inf_map = infection_map(inData)
    inf_map = pd.DataFrame(inf_map).T

    rets,rets, degs = dict(), dict(), dict()

    for i, infector in inf_map.iterrows():
        j=1
        ret = infector[1].copy()
        while len(infector[j]) > 0:
            j+=1
            ret += infector[j]

        rets[i] = ret
        lens[i] = len(ret)
        degs[i] = j
    inf_map['all_infected'] = pd.Series(rets)   
    inf_map['n_infected'] = pd.Series(lens)  
    inf_map['degree'] = pd.Series(degs)  
    return inf_map
inf_map = read_population()

In [382]:
def plot_heat(data, radius = 15, blur = 5, min_opacity = 0.5, gradient = None):
    map_osm = folium.Map(location=CENTER,
                     zoom_start=12.5,tiles='StamenTerrain',control_scale=True)
    hm_wide = HeatMap( [_[:-1] for _ in data],max_value = 20, 
                       min_opacity=min_opacity,
                       radius=radius, blur=blur, 
                       max_zoom=1, gradient = gradient
                     )
    map_osm.add_child(hm_wide)
    feature_group = folium.FeatureGroup("Locations")
    
    for point in data:
        feature_group.add_child(folium.map.Marker([point[0],point[1]],icon=DivIcon(icon_size=(15,36),icon_anchor=(0,0),
            html='<div style="font-size: 10pt">%s</div>' % str(point[3]),
            )))
    map_osm.add_child(feature_group)
    return map_osm 

In [392]:
def plot_infection_one(infector):
    row = inf_map.loc[infector]
    data = [[inData.requests.loc[infector].oy,inData.requests.loc[infector].ox, 1,0]]
    for i in range(1,row.degree):    
        for infected in row[i]:
            data.append([inData.requests.loc[infected].oy,inData.requests.loc[infected].ox, 1- i/row.degree, i])
    return plot_heat(data, blur = 10)
infector = inf_map.sort_values('n_infected', ascending = False).iloc[1].name
plot_infection_one(infector)

In [12]:
inData.requests['ox'] = inData.requests.apply(lambda r: inData.nodes.loc[r.origin].x, axis =1)
inData.requests['oy'] = inData.requests.apply(lambda r: inData.nodes.loc[r.origin].y, axis =1)
inData.requests['dx'] = inData.requests.apply(lambda r: inData.nodes.loc[r.destination].x, axis =1)
inData.requests['dy'] = inData.requests.apply(lambda r: inData.nodes.loc[r.destination].y, axis =1)

In [381]:
path = "res/population_3000_init_0.025_repl_1584525041.2557492.csv"
df = pd.read_csv(path)
del df['Unnamed: 0']
df['ox'] = inData.requests.ox
df['oy'] = inData.requests.oy
df['dx'] = inData.requests.dx
df['dy'] = inData.requests.dy
df = df[df.infection_day>-5]
df.infection_day = df.infection_day.apply(lambda x: 0 if x<0 else x+1)
df['heat'] = 1-1/(df.infection_day.astype(float)+1)
data = list(zip(df.dy.values, df.dx.values, df.heat.values, df.infection_day.astype(int).values))
plot_heat(data)